# hypertension drug choice

## trait

In [ ]:
data = read.table("hypertension_srWGS_person.txt", header=TRUE, sep="\t")
data = data[, "person_id", drop=FALSE]
data$IID = data$person_id
dim(data)

In [ ]:
for (i in c("C02", "C03", "C07", "C08", "C09")) {
#    foo = read.table(paste0("hypertension_srWGS_", i, ".txt"), header=TRUE, sep="\t")
    foo = read.table(paste0("hypertension_srWGS_", i, "_3months.txt"), header=TRUE, sep="\t")
    data[, i] = (data$person_id %in% foo$person_id) * 1
}

In [ ]:
# onlyaffected
foo = read.table("hypertension_srWGS_hypertensivedisorder.txt", header=TRUE)
data = data[data$person_id %in% foo$person_id, ]

In [ ]:
colnames(data)[1] = "FID"

In [ ]:
summary(data)

In [ ]:
cor(data[, -c(1:2)])

In [ ]:
table(rowSums(data[, -c(1:2)]))

In [ ]:
write.table(
    data,
    sep=" ",
    quote=FALSE,
    row.names=FALSE,
#    file = "aou_hypertensiondrug_BT.txt")
    file = "aou_hypertensiondrug_onlyaffected_BT_3months.txt")

In [ ]:
# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./aou_hypertensiondrug_onlyaffected_BT_3months.txt", " ", my_bucket, "/data/"), intern=T)

## covariates

In [ ]:
library(tidyverse)

In [ ]:
data = read.table("hypertension_srWGS_person.txt", header=TRUE, sep="\t")
data$IID = data$person_id
data = data[, c(1, 4, 2, 3)]
dim(data)

The CDRv8 for both tiers (Controlled Tier C2024Q3R4 and Registered Tier R2024Q3R3) includes participant data with a cutoff date of October 1, 2023.

In [ ]:
data$age =
    round(as.numeric(
        with_tz(ymd(20231001), "UTC") -
        ymd_hms(data$date_of_birth)) / 365.2422)

In [ ]:
data$agec2 = round((data$age - mean(data$age, na.rm=TRUE))^2, digits=2)

In [ ]:
foo = read.table("hypertension_srWGS_BMI.txt", header=TRUE, sep="\t")
data$bmi = foo$BMI[match(data$person_id, foo$person_id)]

In [ ]:
system("gsutil -u $GOOGLE_PROJECT cp gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv .", intern=T)

In [ ]:
Anc <- read_tsv('ancestry_preds.tsv')
dim(Anc)
head(Anc)

In [ ]:
Anc_str <- data.frame(str_split_fixed(Anc$pca_features, ', ', 16))
colnames(Anc_str) <- paste0("PC", 1:16)
head(Anc_str)

In [ ]:
Anc_str$PC1 = sub("\\[", "", Anc_str$PC1)
Anc_str$PC16 = sub("\\]", "", Anc_str$PC16)
head(Anc_str)

In [ ]:
for (i in 1:ncol(Anc_str)) {
    Anc_str[, i] = as.numeric(Anc_str[, i])
}
head(Anc_str)

In [ ]:
table(Anc$ancestry_pred)

In [ ]:
Anc_str[Anc$ancestry_pred != "eur", ] = NA
head(Anc_str)
summary(Anc_str)

In [ ]:
data = cbind(
    data,
    Anc_str[match(data$person_id, Anc$research_id), ])

In [ ]:
head(data)

In [ ]:
colnames(data)[1] = "FID"
data = data[, colnames(data) != "date_of_birth"]
head(data)

In [ ]:
summary(data)

In [ ]:
# onlyaffected
foo = read.table("hypertension_srWGS_hypertensivedisorder.txt", header=TRUE)
data = data[data$IID %in% foo$person_id, ]

In [ ]:
write.table(
    data,
    sep= "\t",
    quote=FALSE,
    row.names=FALSE,
    file="aou_hypertensiondrug_onlyaffected_covariates.txt")

In [ ]:
# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./aou_hypertensiondrug_onlyaffected_covariates.txt", " ", my_bucket, "/data/"), intern=T)

## check for Table 1

In [ ]:
data = read.table("aou_hypertensiondrug_onlyaffected_BT_3months.txt", header=T)
foo = read.table("aou_hypertensiondrug_onlyaffected_covariates.txt", header=T, sep="\t")[, 1:7]

In [ ]:
data = cbind(data, foo[match(data$FID, foo$FID), -c(1:2)])

In [ ]:
library(bigrquery)
library(tidyverse)

download_data <- function(query) {
tb <- bq_project_query(Sys.getenv('GOOGLE_PROJECT'), query = query, default_dataset = Sys.getenv('WORKSPACE_CDR'))
bq_table_download(tb)
}


df = download_data(str_glue("SELECT DISTINCT
person_id,
MIN(observation_date) AS primary_consent_date
FROM `concept`
JOIN `concept_ancestor` on concept_id = ancestor_concept_id
JOIN `observation` on descendant_concept_id = observation_source_concept_id
WHERE concept_name = 'Consent PII' AND concept_class_id = 'Module'
GROUP BY 1"))
head(df)

In [ ]:
foo = read.table("hypertension_srWGS_person.txt", header=T, sep="\t")
foo$primary_consent_date = df$primary_consent_date[match(foo$person_id, df$person_id)]
foo$age =
    round(as.numeric(
        foo$primary_consent_date -
        as.Date(ymd_hms(foo$date_of_birth))) / 365.2422)
data$agepcd = foo$age[match(data$IID, foo$person_id)]

In [ ]:
foo = read.table("diabetes_srWGS_diabetesmellitus.txt", header=T, sep="\t")
data$diabetesmellitus = data$IID %in% foo$person_id

In [ ]:
foo = read.table("lipidaemia_srWGS_withC10B_disorderlipid.txt", header=T, sep="\t")
data$disorderlipid = data$IID %in% foo$person_id

In [ ]:
foo = read.table("hypertension_srWGS_ischemicheartdisease.txt", header=T, sep="\t")
data$ischemicheartdisease = data$IID %in% foo$person_id

In [ ]:
# eur
data = data[!is.na(data$PC1), ]

In [ ]:
output = matrix(
    c(nrow(data), NA,
      sum(data$sexM, na.rm=T), sum(data$sexM, na.rm=T)/sum(!is.na(data$sexM)),
      mean(data$agepcd, na.rm=T), sd(data$agepcd, na.rm=T),
      mean(data$bmi, na.rm=T), sd(data$bmi, na.rm=T),
     sum(data$diabetesmellitus), sum(data$diabetesmellitus)/nrow(data),
     sum(data$disorderlipid), sum(data$disorderlipid)/nrow(data),
     sum(data$ischemicheartdisease), sum(data$ischemicheartdisease)/nrow(data)),
    ncol=2, byrow=T)

In [ ]:
output = rbind(output,
cbind(
    colSums(data[, c(3:7)]),
    colSums(data[, c(3:7)])/nrow(data)))

In [ ]:
output

In [ ]:
write.csv(output, file="foo.csv")

# lipidaemia drug choice

## trait

In [ ]:
# data = read.table("lipidaemia_srWGS_person.txt", header=TRUE, sep="\t")
data = read.table("lipidaemia_srWGS_withC10B_person.txt", header=TRUE, sep="\t")
data = data[, "person_id", drop=FALSE]
data$IID = data$person_id
dim(data)

In [ ]:
for (i in c("C10AA", "C10AB", "C10AC", "C10AD", "C10AX06", "C10AX09")) {
    foo = read.table(paste0("lipidaemia_srWGS_withC10B_", i, "_3months.txt"), header=TRUE, sep="\t")
    data[, i] = (data$person_id %in% foo$person_id) * 1
}

In [ ]:
# onlyaffected
foo = read.table("lipidaemia_srWGS_withC10B_disorderlipid.txt", header=TRUE)
data = data[data$person_id %in% foo$person_id, ]

In [ ]:
colnames(data)[1] = "FID"

In [ ]:
summary(data)

In [ ]:
cor(data[, -c(1:2)])

In [ ]:
table(rowSums(data[, -c(1:2)]))

In [ ]:
colSums(data[, -c(1:2)])

In [ ]:
write.table(
    data,
    sep=" ",
    quote=FALSE,
    row.names=FALSE,
    file = "aou_lipidaemiadrug_withC10B_BT_3months.txt")

In [ ]:
# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./aou_lipidaemiadrug_withC10B_BT_3months.txt", " ", my_bucket, "/data/"), intern=T)

## covariates

In [ ]:
library(tidyverse)

In [ ]:
# data = read.table("lipidaemia_srWGS_person.txt", header=TRUE, sep="\t")
data = read.table("lipidaemia_srWGS_withC10B_person.txt", header=TRUE, sep="\t")
data$IID = data$person_id
data = data[, c(1, 4, 2, 3)]
dim(data)

The CDRv8 for both tiers (Controlled Tier C2024Q3R4 and Registered Tier R2024Q3R3) includes participant data with a cutoff date of October 1, 2023.

In [ ]:
data$age =
    round(as.numeric(
        with_tz(ymd(20231001), "UTC") -
        ymd_hms(data$date_of_birth)) / 365.2422)

In [ ]:
data$agec2 = round((data$age - mean(data$age, na.rm=TRUE))^2, digits=2)

In [ ]:
# foo = read.table("lipidaemia_srWGS_BMI.txt", header=TRUE, sep="\t")
foo = read.table("lipidaemia_srWGS_withC10B_BMI.txt", header=TRUE, sep="\t")
data$bmi = foo$BMI[match(data$person_id, foo$person_id)]

In [ ]:
system("gsutil -u $GOOGLE_PROJECT cp gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv .", intern=T)

In [ ]:
Anc <- read_tsv('ancestry_preds.tsv')
dim(Anc)
head(Anc)

In [ ]:
Anc_str <- data.frame(str_split_fixed(Anc$pca_features, ', ', 16))
colnames(Anc_str) <- paste0("PC", 1:16)
head(Anc_str)

In [ ]:
Anc_str$PC1 = sub("\\[", "", Anc_str$PC1)
Anc_str$PC16 = sub("\\]", "", Anc_str$PC16)
head(Anc_str)

In [ ]:
for (i in 1:ncol(Anc_str)) {
    Anc_str[, i] = as.numeric(Anc_str[, i])
}
head(Anc_str)

In [ ]:
table(Anc$ancestry_pred)

In [ ]:
Anc_str[Anc$ancestry_pred != "eur", ] = NA
head(Anc_str)
summary(Anc_str)

In [ ]:
data = cbind(
    data,
    Anc_str[match(data$person_id, Anc$research_id), ])

In [ ]:
head(data)

In [ ]:
colnames(data)[1] = "FID"
data = data[, colnames(data) != "date_of_birth"]
head(data)

In [ ]:
summary(data)

In [ ]:
# onlyaffected
foo = read.table("lipidaemia_srWGS_withC10B_disorderlipid.txt", header=TRUE)
data = data[data$IID %in% foo$person_id, ]

In [ ]:
write.table(
    data,
    sep= "\t",
    quote=FALSE,
    row.names=FALSE,
#    file="aou_lipidaemiadrug_covariates.txt")
    file="aou_lipidaemiadrug_withC10B_covariates.txt")

In [ ]:
# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
# system(paste0("gsutil cp ./aou_lipidaemiadrug_covariates.txt", " ", my_bucket, "/data/"), intern=T)
system(paste0("gsutil cp ./aou_lipidaemiadrug_withC10B_covariates.txt", " ", my_bucket, "/data/"), intern=T)

## check for Table 1

In [ ]:
data = read.table("aou_lipidaemiadrug_withC10B_BT_3months.txt", header=T)
foo = read.table("aou_lipidaemiadrug_withC10B_covariates.txt", header=T, sep="\t")[, 1:7]

In [ ]:
data = cbind(data, foo[match(data$FID, foo$FID), -c(1:2)])

In [ ]:
library(bigrquery)
library(tidyverse)

download_data <- function(query) {
tb <- bq_project_query(Sys.getenv('GOOGLE_PROJECT'), query = query, default_dataset = Sys.getenv('WORKSPACE_CDR'))
bq_table_download(tb)
}


df = download_data(str_glue("SELECT DISTINCT
person_id,
MIN(observation_date) AS primary_consent_date
FROM `concept`
JOIN `concept_ancestor` on concept_id = ancestor_concept_id
JOIN `observation` on descendant_concept_id = observation_source_concept_id
WHERE concept_name = 'Consent PII' AND concept_class_id = 'Module'
GROUP BY 1"))
head(df)

In [ ]:
foo = read.table("lipidaemia_srWGS_withC10B_person.txt", header=T, sep="\t")
foo$primary_consent_date = df$primary_consent_date[match(foo$person_id, df$person_id)]
foo$age =
    round(as.numeric(
        foo$primary_consent_date -
        as.Date(ymd_hms(foo$date_of_birth))) / 365.2422)
data$agepcd = foo$age[match(data$IID, foo$person_id)]

In [ ]:
foo = read.table("diabetes_srWGS_diabetesmellitus.txt", header=T, sep="\t")
data$diabetesmellitus = data$IID %in% foo$person_id

In [ ]:
foo = read.table("hypertension_srWGS_hypertensivedisorder.txt", header=T, sep="\t")
data$hypertensivedisorder = data$IID %in% foo$person_id

In [ ]:
foo = read.table("lipidaemia_srWGS_ischemicheartdisease.txt", header=T, sep="\t")
data$ischemicheartdisease = data$IID %in% foo$person_id

In [ ]:
# eur
data = data[!is.na(data$PC1), ]

In [ ]:
output = matrix(
    c(nrow(data), NA,
      sum(data$sexM, na.rm=T), sum(data$sexM, na.rm=T)/sum(!is.na(data$sexM)),
      mean(data$agepcd, na.rm=T), sd(data$agepcd, na.rm=T),
      mean(data$bmi, na.rm=T), sd(data$bmi, na.rm=T),
     sum(data$diabetesmellitus), sum(data$diabetesmellitus)/nrow(data),
     sum(data$hypertensivedisorder), sum(data$hypertensivedisorder)/nrow(data),
     sum(data$ischemicheartdisease), sum(data$ischemicheartdisease)/nrow(data)),
    ncol=2, byrow=T)

In [ ]:
output = rbind(output,
cbind(
    colSums(data[, c(3:8)]),
    colSums(data[, c(3:8)])/nrow(data)))

In [ ]:
output

In [ ]:
write.csv(output, file="foo.csv")

# drug response

## trait

In [ ]:
myboxcox1 = function (x, l=1) {
  if (l==0) {
    return(log(x))
  } else {
    return((x^l - 1)/l)
  }
}

myboxcox2 = function (x, y, lambda=seq(-2, 2, 1/10)) {
  coef = as.numeric(lapply(
    lambda,
    function (l) {
      df = data.frame(x=myboxcox1(x, l), y=myboxcox1(y, l))
    #  return(sum((df$y - mean(df$y) - df$x + mean(df$x))^2) /
    #           sum((df$x - mean(df$x))^2 + (df$y - mean(df$y))^2))
    # 2025.09.11 myboxcoxv2
      return(abs(log(sum((df$x - mean(df$x))^2) /
                       sum((df$y - mean(df$y))^2))))
    }))
  print(coef)
  return(lambda[which.min(coef)])
}

In [ ]:
library(tidyverse)

data4 = read.table(
#  "antihypertensive.SBP.minafterstart28.wpower1.txt",
  "hypolipidemics.LDL.minafterstart28.wpower1.txt",
#    "antidiabetic.HbA1c.wpower1.txt",
  sep="\t",
  header=TRUE)

# only for plotting LDL; convert mg/dL to mmol/L
# actually, also for GWAS
data4 = data4 %>%
  mutate(value.drugfree = 0.02586 * value.drugfree,
         value.drugtake = 0.02586 * value.drugtake)

data5 = data4 %>%
  group_by(drug) %>%
  mutate(value.drugfree2 = (value.drugfree - mean(value.drugfree))^2) %>%
  ungroup() %>%
  mutate(residual = NA, myboxcox=NA)
for (d in unique(data5$drug)) {
  x = (data5$drug == d)
  a0 = lm(value.drugtake ~ value.drugfree + value.drugfree2, data=data5, subset=x)
  data5$residual[x] = resid(a0)
}
for (d in unique(data5$drug)) {
  x = (data5$drug == d)
  l = myboxcox2(data5$value.drugfree[x], data5$value.drugtake[x],
               #lambda=seq(0, 1, 1/20)) #myboxcox01
               lambda=seq(-2, 2, 1/20)) #myboxcoxv2
  print(paste(d, sum(x), l))
  data5$myboxcox[x] = myboxcox1(data5$value.drugtake[x], l) - myboxcox1(data5$value.drugfree[x], l)
}

output =
  data5 %>%
  select(person_id, drug, residual, myboxcox, value.drugfree, value.drugtake, value.delta) %>%
  group_by(person_id, drug) %>%
  summarize(value =
#            mean(residual),
            mean(myboxcox),
#            mean(value.delta), #LDLdelta
#            mean(log(value.drugtake/value.drugfree)), #LDLlogratio
            .groups="drop") %>%
  pivot_wider(names_from=drug, values_from=value) %>%
  arrange(person_id) %>%
  rename(IID=person_id) %>%
  mutate(FID=IID) %>%
  relocate(FID, .before=IID)

In [ ]:
write.table(
  output,
  #file="aou_antihypertensive.SBP_QT.txt",
  #file="aou_antihypertensive.SBPmyboxcox_QT.txt",
  #file="aou_antihypertensive.SBPmyboxcox01_QT.txt",
  #file="aou_antihypertensive.SBPmyboxcoxv2_QT.txt",
  #file="aou_hypolipidemics.LDL_QT.txt",
  #file="aou_hypolipidemics.LDLmyboxcox_QT.txt",
  #file="aou_hypolipidemics.LDLmyboxcox01_QT.txt",
  file="aou_hypolipidemics.LDLmyboxcoxv2_QT.txt",
  #file="aou_hypolipidemics.LDLdelta_QT.txt",
  #file="aou_hypolipidemics.LDLlogratio_QT.txt",
  #file="aou_antidiabetic.HbA1c_QT.txt",
  #file="aou_antidiabetic.HbA1cmyboxcox_QT.txt",
  #file="aou_antidiabetic.HbA1cmyboxcox01_QT.txt",
#  output2,
#  file="aou_antidiabetic.HbA1c_QT.FID0.txt",
  sep=" ",
  quote=FALSE,
  row.names=FALSE,
  na="NA"
)

In [ ]:
# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0(
#    "gsutil cp ./aou_antihypertensive.SBP_QT.txt",
#    "gsutil cp ./aou_antihypertensive.SBPmyboxcox_QT.txt",
#    "gsutil cp ./aou_antihypertensive.SBPmyboxcox01_QT.txt",
#    "gsutil cp ./aou_hypolipidemics.LDL_QT.txt",
#    "gsutil cp ./aou_hypolipidemics.LDLmyboxcox_QT.txt",
#    "gsutil cp ./aou_hypolipidemics.LDLmyboxcox01_QT.txt",
    "gsutil cp ./aou_hypolipidemics.LDLmyboxcoxv2_QT.txt",
#    "gsutil cp ./aou_hypolipidemics.LDLdelta_QT.txt",
#    "gsutil cp ./aou_hypolipidemics.LDLlogratio_QT.txt",
#    "gsutil cp ./aou_antidiabetic.HbA1c_QT.txt",
#    "gsutil cp ./aou_antidiabetic.HbA1c_QT.FID0.txt",
#    "gsutil cp ./aou_antidiabetic.HbA1cmyboxcox_QT.txt",
#    "gsutil cp ./aou_antidiabetic.HbA1cmyboxcox01_QT.txt",
    " ", my_bucket, "/data/"), intern=T)

## covariates

In [ ]:
library(tidyverse)

In [ ]:
data = output[, 1:2]
dim(data)

In [ ]:
system("gsutil -u $GOOGLE_PROJECT cp gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv .", intern=T)

In [ ]:
Anc <- read_tsv('ancestry_preds.tsv')
dim(Anc)
head(Anc)

In [ ]:
Anc_str <- data.frame(str_split_fixed(Anc$pca_features, ', ', 16))
colnames(Anc_str) <- paste0("PC", 1:16)
head(Anc_str)

In [ ]:
Anc_str$PC1 = sub("\\[", "", Anc_str$PC1)
Anc_str$PC16 = sub("\\]", "", Anc_str$PC16)
head(Anc_str)

In [ ]:
for (i in 1:ncol(Anc_str)) {
    Anc_str[, i] = as.numeric(Anc_str[, i])
}
head(Anc_str)

In [ ]:
table(Anc$ancestry_pred)

In [ ]:
Anc_str[Anc$ancestry_pred != "eur", ] = NA
head(Anc_str)
summary(Anc_str)

In [ ]:
data = cbind(
    data,
    Anc_str[match(data$IID, Anc$research_id), ])

In [ ]:
head(data)

In [ ]:
summary(data)

In [ ]:
data2 = data
data2$FID = 0

In [ ]:
write.table(
    data,
#    file="aou_antihypertensive.SBP_covariates.txt",
#    file="aou_hypolipidemics.LDL_covariates.txt",
#    file="aou_antidiabetic.HbA1c_covariates.txt",
#    data2,
#    file="aou_antidiabetic.HbA1c_covariates.FID0.txt",
    sep= "\t",
    quote=FALSE,
    row.names=FALSE
)

In [ ]:
# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0(
#    "gsutil cp ./aou_antihypertensive.SBP_covariates.txt",
#    "gsutil cp ./aou_hypolipidemics.LDL_covariates.txt",
#    "gsutil cp ./aou_antidiabetic.HbA1c_covariates.txt",
#    "gsutil cp ./aou_antidiabetic.HbA1c_covariates.FID0.txt",
    " ", my_bucket, "/data/"), intern=T)

In [ ]:
dataplot = data5 %>%
  filter(drug=="C10AA") %>%
  mutate(deltaboxcox0 = myboxcox1(value.drugtake, 0) - myboxcox1(value.drugfree, 0),
         deltaboxcox1 = myboxcox1(value.drugtake, 1) - myboxcox1(value.drugfree, 1))

dataplot$PC1 = data$PC1[match(dataplot$person_id, data$IID)]
dataplot = dataplot[!is.na(dataplot$PC1), ]

library(GGally)
f = ggpairs(dataplot,
        columns=c("value.drugfree", "value.drugtake", "deltaboxcox1", "myboxcox", "deltaboxcox0"),
        lower = list(continuous = wrap(ggally_points, size=0.1, alpha=0.1)),
        diag = list(continuous = "blankDiag"))

In [ ]:
ggsave(f, file="ggpair_aou_C10AA_myboxcox015.pdf")

In [ ]:
head(dataplot)

In [ ]:
foo = read.table("rs7412.raw", header=TRUE)
dataplot$rs7412_T = foo$`chr19.44908822.C.T_T`[match(dataplot$person_id, foo$IID)]
dataplot$rs7412_T = factor(dataplot$rs7412_T,
                           labels = c("CC", "CT", "TT"))

dataplot = dataplot[!is.na(dataplot$rs7412_T), ]
dataplot = dataplot[order(dataplot$person_id), ]

In [ ]:
df = data.frame(
  # x = dataplot$value.drugfree,
  # y = dataplot$value.drugtake,
  # x=log(dataplot$value.drugfree),
  # y=log(dataplot$value.drugtake),
   x=myboxcox1(dataplot$value.drugfree, 0.3),
   y=myboxcox1(dataplot$value.drugtake, 0.3),
  rs7412_T = dataplot$rs7412_T)

level <- 0.95
cfac  <- sqrt(qchisq(level, df = 2))

seg_df <- df %>%
  group_by(rs7412_T) %>%
  summarize(
    mx = mean(x), my = mean(y),
    cov_xx = var(x), cov_yy = var(y),
    cov_xy = cov(x, y),
    .groups = "drop"
  ) %>%
  rowwise() %>%
  mutate(
    {S <- matrix(c(cov_xx, cov_xy, cov_xy, cov_yy), 2, 2)
    eig <- eigen(S)
    vx = eig$vectors[1,1]
    vy = eig$vectors[2,1]
    a  <- cfac * sqrt(eig$values[1])
    tibble(vx = vx, vy = vy, a = a)}
  ) %>%
  ungroup()

f = ggplot(df, aes(x, y)) +
  geom_point(size=0.5, alpha=0.5, shape=16) +
  geom_abline(slope=1, intercept=0) +
  stat_ellipse(type = "norm", level = level, linewidth = 0.8, color = "deepskyblue") +
  geom_segment(
    data = seg_df,
    aes(x = mx - a*vx, y = my - a*vy,
        xend = mx + a*vx, yend = my + a*vy),
    inherit.aes = FALSE,
    linewidth = 0.8, color = "deepskyblue"
  ) +
  geom_point(
    data = seg_df,
    aes(x = mx, y = my),
    inherit.aes = FALSE,
    color = "deepskyblue"
  ) +
  coord_fixed(ratio = 1) +
  facet_grid(cols = vars(rs7412_T)) +
  # xlab("LDL drug free") +
  # ylab("LDL under statin")
  # xlab("log(LDL) drug free") +
  # ylab("log(LDL) under statin")
   xlab("Transformed LDL (l=0.3) drug free") +
   ylab("Transformed LDL (l=0.3) under statin")

In [ ]:
ggsave(
#    "drugfree_drugtake_raw.pdf",
#    "drugfree_drugtake_log.pdf",
#    "drugfree_drugtake_boxcox015.pdf",
    "drugfree_drugtake_boxcox03.pdf",
    f, width=6.1, height=3.1)

## check for Table 1

In [ ]:
library(tidyverse)

In [ ]:
zoo = read.table("aou_antihypertensive.SBPmyboxcox01_covariates.txt", header=T)

In [ ]:
summary(zoo)

In [ ]:
#data = read.table("aou_antihypertensive.SBPmyboxcox_QT.txt", header=T)
#foo = read.table("aou_antihypertensive.SBPmyboxcox01_covariates.txt", header=T)
data = read.table("aou_hypolipidemics.LDLmyboxcox01_QT.txt", header=T)
foo = read.table("aou_hypolipidemics.LDLmyboxcox01_covariates.txt", header=T)
data$PC1 = foo$PC1[match(data$FID, foo$FID)]
data = data[!is.na(data$PC1), ]

In [ ]:
#foo = read.table("DBPSBP_srWGS_person.txt", header=T, sep="\t")
#goo = read.table("DBPSBP_srWGS_BMI.txt", header=T, sep="\t")
foo = read.table("HDLLDL_srWGS_person.txt", header=T, sep="\t")
goo = read.table("HDLLDL_srWGS_BMI.txt", header=T, sep="\t")
data[, c("sexM", "date_of_birth")] = foo[match(data$FID, foo$person_id),  c("sexM", "date_of_birth")]
data$BMI = goo$BMI[match(data$FID, goo$person_id)]

The CDRv8 for both tiers (Controlled Tier C2024Q3R4 and Registered Tier R2024Q3R3) includes participant data with a cutoff date of October 1, 2023.

In [ ]:
data$age =
    round(as.numeric(
        with_tz(ymd(20231001), "UTC") -
        ymd_hms(data$date_of_birth)) / 365.2422)

In [ ]:
summary(data)

In [ ]:
output = matrix(
    c(nrow(data), NA,
      sum(data$sexM, na.rm=T), sum(data$sexM, na.rm=T)/sum(!is.na(data$sexM)),
      mean(data$age, na.rm=T), sd(data$age, na.rm=T),
      mean(data$BMI, na.rm=T), sd(data$BMI, na.rm=T)),
    ncol=2, byrow=T)

In [ ]:
output = rbind(output,
cbind(
    colSums(!is.na(data[, seq(3, ncol(data)-5)])),
    colSums(!is.na(data[, seq(3, ncol(data)-5)]))/nrow(data)))

In [ ]:
output

In [ ]:
write.csv(output, file="foo.csv")